This code works through the GEDI subsetter version 0.6.0. This converts a geojson of tiles over a study area to temp geojson files to feed into the subsetter

In [32]:
#Open the MAAP host
from maap.maap import MAAP
maap = MAAP(maap_host='adpe.ops.maap-project.org')


Unable to load config file from source maap.cfg 
Unable to load config file from source ./maap.cfg 
Unable to load config file from source /projects/maap.cfg 


In [33]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import geopandas as gpd
import shapely

In [34]:
#Load in Tiled AOI
tiles = gpd.read_file("https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/WestAfricaGrid.geojson")

print(tiles.info)

<bound method DataFrame.info of          id    random                                           geometry
0      8,-7  0.685510  POLYGON ((8.00000 -7.00000, 9.00000 -7.00000, ...
1      9,-7  0.452251  POLYGON ((9.00000 -7.00000, 10.00000 -7.00000,...
2     12,-7  0.231314  POLYGON ((12.00000 -7.00000, 13.00000 -7.00000...
3      8,-6  0.597992  POLYGON ((8.00000 -6.00000, 9.00000 -6.00000, ...
4      9,-6  0.995462  POLYGON ((9.00000 -6.00000, 10.00000 -6.00000,...
..      ...       ...                                                ...
146  -17,20  0.612239  POLYGON ((-17.00000 20.00000, -16.00000 20.000...
147  -16,20  0.167271  POLYGON ((-16.00000 20.00000, -15.00000 20.000...
148  -18,21  0.473960  POLYGON ((-18.00000 21.00000, -17.00000 21.000...
149  -17,21  0.156771  POLYGON ((-17.00000 21.00000, -16.00000 21.000...
150  -16,21  0.512777  POLYGON ((-16.00000 21.00000, -15.00000 21.000...

[151 rows x 3 columns]>


In [35]:
#Set index to the correct column name, in this case "random"
tiles.set_index('random',inplace=True)
print(tiles.head())

             id                                           geometry
random                                                            
0.685510   8,-7  POLYGON ((8.00000 -7.00000, 9.00000 -7.00000, ...
0.452251   9,-7  POLYGON ((9.00000 -7.00000, 10.00000 -7.00000,...
0.231314  12,-7  POLYGON ((12.00000 -7.00000, 13.00000 -7.00000...
0.597992   8,-6  POLYGON ((8.00000 -6.00000, 9.00000 -6.00000, ...
0.995462   9,-6  POLYGON ((9.00000 -6.00000, 10.00000 -6.00000,...


In [43]:
for col in tiles.columns:
    print(col)

id
geometry


In [44]:
tiles['random_value'] = tiles.index

In [45]:
#Test a single tile
tiles.loc[0.6855097362903453]

id                                                  8,-7
geometry        POLYGON ((8 -7, 9 -7, 9 -6, 8 -6, 8 -7))
random_value                                     0.68551
Name: 0.6855097362903453, dtype: object

Use "random" column for ID to generate features to tile over

In [46]:
#List out the individual aois
list_aois = tiles.index.unique().tolist() #tiles[tiles.columns[1]].values.tolist()
print(list_aois)

[0.6855097362903453, 0.4522512836485828, 0.23131388830190724, 0.5979918962060293, 0.9954619410018374, 0.7363462239298356, 0.16955097115328577, 0.17510753017660985, 0.3925692959970333, 0.5752152674405157, 0.7412455810081872, 0.7392697493154132, 0.4653903570931611, 0.8354018764581234, 0.38600958865659807, 0.23490126284236712, 0.6936479469170777, 0.895062222650941, 0.789772170178162, 0.5160985973254504, 0.9244779703796152, 0.5150738620210101, 0.894109796377685, 0.06989256159489654, 0.6310652493506097, 0.8557841478391265, 0.3657585583436155, 0.7485440110765288, 0.5672959857440157, 0.33153070200335955, 0.39138004097717916, 0.5822984975127008, 0.4411987903729985, 0.5549448029680308, 0.1657510400319916, 0.658522680224497, 0.5666193031784508, 0.4415834231141531, 0.9917320562143129, 0.5326880936290095, 0.9621639521726443, 0.07476782962538708, 0.7317576801263533, 0.8968646730286827, 0.14280037807636703, 0.582712155141294, 0.03319394615667215, 0.6036670510954576, 0.023375478336133515, 0.175617543

In [38]:
aoiTest=shapely.to_geojson(tiles.loc[0.6855097362903453,"geometry"])
print(aoiTest)

{"type":"Polygon","coordinates":[[[8.0,-7.0],[9.0,-7.0],[9.0,-6.0],[8.0,-6.0],[8.0,-7.0]]]}


In [39]:
#Create empty variables to fill with all 29 appearances of cover_z* and pai_z* 
variables = []
for n in np.arange(0, 30,1):
    variables.append('cover_z' + str(n))
    variables.append('pai_z' + str(n))


For several AOI's run this code:

In [40]:
#Set up job results to print output during run

import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [41]:
#Import geojson and tempfile for creating temp files for the subsetter

from geojson import Point, Feature, FeatureCollection, dump

import tempfile

In [128]:
#def write_json(shape):
   # feature is a shapely geometry type
#        geom_in_geojson = geojson.Feature(geometry=shape, properties={})
#        tmp_file = tempfile.mkstemp(suffix='.geojson')
#        with open(tmp_file[1], 'w') as outfile:
#            geojson.dump(geom_in_geojson, outfile)
#        return tmp_file[1]

In [140]:
#list_aois = tiles['random'].unique().tolist()
#list_aois

In [142]:
#for aoi in list_aois[:1]:
    #shape = shapely.to_geojson(tiles.loc[aoi,"geometry"])
    #f = write_json(shape)
#    tile = tiles[tiles['random']==aoi]
    #print(type(tile))
#    tile.to_file('temp.geojson')
    

<class 'geopandas.geodataframe.GeoDataFrame'>


In [53]:
tile = tiles[tiles['random_value']==0.6855097362903453]
tile.to_file('temp.geojson')
print(tiles.head())

             id                                           geometry  \
random                                                               
0.685510   8,-7  POLYGON ((8.00000 -7.00000, 9.00000 -7.00000, ...   
0.452251   9,-7  POLYGON ((9.00000 -7.00000, 10.00000 -7.00000,...   
0.231314  12,-7  POLYGON ((12.00000 -7.00000, 13.00000 -7.00000...   
0.597992   8,-6  POLYGON ((8.00000 -6.00000, 9.00000 -6.00000, ...   
0.995462   9,-6  POLYGON ((9.00000 -6.00000, 10.00000 -6.00000,...   

          random_value  
random                  
0.685510      0.685510  
0.452251      0.452251  
0.231314      0.231314  
0.597992      0.597992  
0.995462      0.995462  


In [50]:
#Set up input dictionary and import backoff to display run results
#First part of the for loop uses the "random" column to search through the tiles geojson to 
#create temp files one at a time to set the AOI
#################################

for aoi in [list_aois[0]]: 
    
    
    #tile = tiles[tiles['random']==aoi]
    #tile.to_file('temp.geojson')
    
    inputs = dict(
       aoi="https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/MAAP_GEDIAnalysis/temp.geojson",
       doi="L2B",
       lat="geolocation/lat_lowestmode",
       lon="geolocation/lon_lowestmode",
       beams="all",
       columns="rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
       query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
       limit = 1000,
       temporal="-",
       output="-"
    )
    result = maap.submitJob(
        identifier="gedi-subset",
        algo_id="gedi-subset_ubuntu",
        version="0.6.0",
        queue="maap-dps-worker-32gb",
        username="abarenblitt",
    **inputs
    )
    
    job_id = result["job_id"]
    print(job_id)
    
try:
    import backoff
except:
    job_status = job_status_for(job_id)
else:
    # Check job status every 2 minutes
    @backoff.on_predicate(
        backoff.constant,
        lambda status: status not in ["Deleted", "Succeeded", "Failed"],
        interval=120,
    )
    def wait_for_job(job_id: str) -> str:
        return job_status_for(job_id)

    job_status = wait_for_job(job_id)
    job_status

UnboundLocalError: local variable 'r' referenced before assignment

In [52]:
inputs = dict(
   aoi="https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/MAAP_GEDIAnalysis/temp.geojson",
   doi="L2B",
   lat="geolocation/lat_lowestmode",
   lon="geolocation/lon_lowestmode",
   beams="all",
   columns="rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
   query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
   limit = 10_000,
   temporal="-",
   output="-"
)
result = maap.submitJob(
    identifier="gedi-subset",
    algo_id="gedi-subset_ubuntu",
    version="0.6.0",
    queue="maap-dps-worker-32gb",
    username="abarenblitt",
    **inputs
)
inputs

UnboundLocalError: local variable 'r' referenced before assignment

In [144]:
job_id = "9a90ea3d-60a1-4cb2-9b5e-a3886a603fc"

job_status

'Failed'

In [96]:
#Get job result full text to bug Alex with
 
maap.getJobResult(job_id).text

'<wps:Result xmlns:ows="http://www.opengis.net/ows/2.0" xmlns:schemaLocation="http://schemas.opengis.net/wps/2.0/wps.xsd" xmlns:wps="http://www.opengis.net/wps/2.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><wps:JobID>a4596ff3-8bfc-4d03-a9dc-3cb4883b9d59</wps:JobID><wps:Output id="traceback"><wps:Data>Traceback (most recent call last):\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/storage/file.py", line 67, in get\n    fh = open(path, "r")\nFileNotFoundError: [Errno 2] No such file or directory: \'POLYGON ((-16 21, -15 21, -15 22, -16 22, -16 21))\'\n\nDuring handling of the above exception, another exception occurred:\n\nTraceback (most recent call last):\n  File "/home/ops/verdi/ops/hysds-0.3.11/hysds/utils.py", line 139, in download_file\n    osaka.main.get(url, cache_dir, params=params)\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/main.py", line 86, in get\n    noclobber=noclobber,\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/main.py", line 128, in transfer\n    no